In [1]:
import sys
sys.path.append('../sgmm')
sys.path.append('../metrics')
sys.path.append('../loaders')
sys.path.append('../Misc')
sys.path.append('../visual')
sys.path.append('../otherModels')
sys.path.append('../LogOdds')

In [2]:
import numpy as np
import pandas as pd
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from supervisedGmm import SupervisedGMM
from metricsFunctions import calc_metrics, CalculateSoftLogReg, optimalTau,metrics_cluster,sgmmResults
# from loaders2 import loader
# from mlModels import logisticRegressionCv2, neural_nets, randomforests,\
# kmeansLogRegr, xboost, gradboost,kmeansBNB
# from sklearn.naive_bayes import BernoulliNB
# from utility import entropy,asymm_entropy
# from ftest_logodds import ftest_uncorr
# from ftest_logodds import restest
# #from clustmap import plotclustmap
# from clustmap_newborn_kmeans_noopt import plotclustmap
# from sklearn.linear_model import LogisticRegression

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
sparcs = pd.read_csv("~/data/CDPHP/xiao/SPARCS_Subsets/Obsolete/sparcs25%Newborn_DeHos_Outflow_Region.csv") 

#sparcs = sparcs.sample(frac=0.02, random_state = 1512)

d_newborn_tr, d_newborn_te = train_test_split(sparcs, test_size=0.2, random_state = 1512)

print(d_newborn_tr.shape[0], d_newborn_tr.shape[1])

93543 260


In [5]:
# metric matrix columns
columns = ['cluster', 'size', 'high_cost%','low_cost%', 
                       'TP', 'TN', 'FP', 'FN', 
                       'FPR', 'specificity', 'sensitivity', 'precision',
                       'accuracy', 'balanced accuracy', 'f1', 'auc']

In [6]:
# feature list
features = list(sparcs.iloc[:,:-1])

In [7]:
#Preparing the data
Xtrain, Xtest = d_newborn_tr.iloc[:,0:-1].values, d_newborn_te.iloc[:,0:-1].values
ytrain, ytest = d_newborn_tr.iloc[:,-1].values.astype(int), d_newborn_te.iloc[:,-1].values.astype(int)

In [8]:
#VALIDATION ON CLUSTERS
#RUN VALIDATION ON NUMBER OF CLUSTERS BASED ON AUC & F1

import time
import random

train, val = train_test_split(d_newborn_tr, test_size = 0.2, random_state = 1512)
y_train, y_val = train.iloc[:,-1], val.iloc[:,-1]

print( train.shape, val.shape )

#SET SGMM PARAMETERS

max_iter = 30
max_iter2 = 30

train_np = train.iloc[:,0:-1].values
val_np = val.iloc[:,0:-1].values

y_train_np = y_train.values
y_val_np = y_val.values

test_re = []
train_re = []
seed_list = []

start = time.time()

for j in range(0,10): 

    seed_num = random.randint(0,100000)
    seed_list.append(seed_num)
    print("\n TRAINING WITH RANDOM SEED {}". format(seed_num))
#test clusters 2-10
    for i in range(2,11):
        print("\n\n\nTRAINING WITH {} CLUSTERS".format(i))
        np.random.seed( seed = seed_num )
        model = SupervisedGMM(max_iter = max_iter, max_iter2 = max_iter2, n_clusters = i)
        model = model.fit(Xtrain = train_np, ytrain = y_train_np)
        probTrain = model.predict_proba( train_np )
        probTest = model.predict_proba( val_np )
        results = sgmmResults( model, probTest.copy(), probTrain.copy(), y_val_np.copy(), y_train_np.copy(), tau = None,
            mode = 0)
        testmetrics = results['testMet']
        trainMetrics = results[ 'trainMet']
        test_re.append( testmetrics )
        train_re.append( trainMetrics )
    
end = time.time() - start
print("time elapsed: {}".format( end ))

(74834, 260) (18709, 260)

 TRAINING WITH RANDOM SEED 2721



TRAINING WITH 2 CLUSTERS
GMM iteration: 0, error: 0.28733822249316937
GMM iteration: 1, error: 0.30739477633138473
GMM iteration: 2, error: 0.06478305760776436
GMM iteration: 3, error: 0.01651501538522494
GMM iteration: 4, error: 0.004214684767153675
GMM iteration: 5, error: 0.0012168723693190846
GMM iteration: 6, error: 0.00024373655599722093



TRAINING WITH 3 CLUSTERS
GMM iteration: 0, error: 0.42159619641550106
GMM iteration: 1, error: 0.10935846884331285
GMM iteration: 2, error: 0.023909640556135807
GMM iteration: 3, error: 0.0056999878505636535
GMM iteration: 4, error: 0.001267741506112734
GMM iteration: 5, error: 0.00032131007249153616



TRAINING WITH 4 CLUSTERS
GMM iteration: 0, error: 0.37073131844238577
GMM iteration: 1, error: 0.07858078179855442
GMM iteration: 2, error: 0.017079598454180188
GMM iteration: 3, error: 0.0041934879689777575
GMM iteration: 4, error: 0.0009417326878749451



TRAINING WITH 5 CLUSTERS
G

GMM iteration: 1, error: 0.10331597687866573
GMM iteration: 2, error: 0.021590547758342713
GMM iteration: 3, error: 0.00491206865324923
GMM iteration: 4, error: 0.0009819893395490455



TRAINING WITH 3 CLUSTERS
GMM iteration: 0, error: 0.43800545284941866
GMM iteration: 1, error: 0.09514171309248093
GMM iteration: 2, error: 0.022754546549194246
GMM iteration: 3, error: 0.004795927066031652
GMM iteration: 4, error: 0.0009994946642070397



TRAINING WITH 4 CLUSTERS
GMM iteration: 0, error: 0.3644428022448858
GMM iteration: 1, error: 0.08379828626365607
GMM iteration: 2, error: 0.018815920005478277
GMM iteration: 3, error: 0.004370539602046431
GMM iteration: 4, error: 0.0016436543151766092
GMM iteration: 5, error: 0.0009803606447384063



TRAINING WITH 5 CLUSTERS
GMM iteration: 0, error: 0.31076029919063464
GMM iteration: 1, error: 0.0730384196143394
GMM iteration: 2, error: 0.015832469657159925
GMM iteration: 3, error: 0.004495391241921439
GMM iteration: 4, error: 0.0013306267344986788
G

GMM iteration: 3, error: 0.0027545035767855062
GMM iteration: 4, error: 0.0005581650107980995

 TRAINING WITH RANDOM SEED 89082



TRAINING WITH 2 CLUSTERS
GMM iteration: 0, error: 0.35946270224035226
GMM iteration: 1, error: 0.22930988171953326
GMM iteration: 2, error: 0.04858410472307885
GMM iteration: 3, error: 0.010185016265526627
GMM iteration: 4, error: 0.0020494079258336685
GMM iteration: 5, error: 0.00041260977659354596



TRAINING WITH 3 CLUSTERS
GMM iteration: 0, error: 0.27200390652363426
GMM iteration: 1, error: 0.2924483998016384
GMM iteration: 2, error: 0.061030816460077714
GMM iteration: 3, error: 0.012811742492436806
GMM iteration: 4, error: 0.0026694426146437636
GMM iteration: 5, error: 0.0012952102122044972
GMM iteration: 6, error: 0.0007803190823446723



TRAINING WITH 4 CLUSTERS
GMM iteration: 0, error: 0.22907677505306567
GMM iteration: 1, error: 0.2647512648378588
GMM iteration: 2, error: 0.05551225380381417
GMM iteration: 3, error: 0.012839915611810072
GMM iterat




TRAINING WITH 10 CLUSTERS
GMM iteration: 0, error: 0.17118776365946425
GMM iteration: 1, error: 0.04551987906013002
GMM iteration: 2, error: 0.010182425083909402
GMM iteration: 3, error: 0.0023634414563000685
GMM iteration: 4, error: 0.0006734299592812119

 TRAINING WITH RANDOM SEED 85838



TRAINING WITH 2 CLUSTERS
GMM iteration: 0, error: 0.3305399465925064
GMM iteration: 1, error: 0.2590048420356773
GMM iteration: 2, error: 0.05244838914412629
GMM iteration: 3, error: 0.011892027491182563
GMM iteration: 4, error: 0.00309770688045101
GMM iteration: 5, error: 0.0006692159061974676



TRAINING WITH 3 CLUSTERS
GMM iteration: 0, error: 0.33805826168549685
GMM iteration: 1, error: 0.1885568235462176
GMM iteration: 2, error: 0.03961470732325653
GMM iteration: 3, error: 0.009753478417435003
GMM iteration: 4, error: 0.0046655392209426095
GMM iteration: 5, error: 0.001323300517005758
GMM iteration: 6, error: 0.0011151059404450245
GMM iteration: 7, error: 0.00025998759536627304



TRAINING 

In [21]:
#PICK THE CLUSTERS

pdi = pd.DataFrame(np.zeros(shape = [len(test_re), test_re[0].shape[1]]), columns = columns)
pdi_tr = pd.DataFrame(np.zeros(shape = [len(train_re), train_re[0].shape[1]]), columns = columns)

for i, panda in enumerate(test_re):
     pdi.iloc[i,:] = panda.iloc[0,:]
        
for i, panda in enumerate(train_re):
     pdi_tr.iloc[i,:] = panda.iloc[0,:]

pdi['seed_number'] = pd.DataFrame(np.repeat(np.array(seed_list),9) )
pdi['cluster']= pd.DataFrame(np.tile(np.arange(2,11),10))        





In [26]:
print('Here are the number of clusters and seed_number for the largest auc ')
print(pdi.sort_values(by = 'auc',ascending=False).iloc [0,:][['cluster','auc','seed_number']])
print('Here are the number of clusters and seed_number for the smallest auc ')
print(pdi.sort_values(by = 'auc',ascending=True).iloc [0,:][['cluster','auc','seed_number']])

print('BASED ON THAT WE PICK 7 CLUSTERS with seed = 21274')

Here are the number of clusters and seed_number for the largest auc 
cluster            7.000000
auc                0.814116
seed_number    21274.000000
Name: 14, dtype: float64
Here are the number of clusters and seed_number for the smallest auc 
cluster            4.00000
auc                0.80281
seed_number    59412.00000
Name: 29, dtype: float64
BASED ON THAT WE PICK 7 CLUSTERS with seed = 21274
